# EigenSpace Mapping: 53-TET Chord Positions

This notebook maps **53-TET (53 equal temperament)** chords to the **EigenSpace** coordinate system.

## EigenSpace Coordinate System
- **(α, β, γ)** = frequency ratios of the 2nd, 3rd, and 4th notes relative to the root
- **Constraint**: α ≤ β ≤ γ (tetrahedron region, eliminates redundant permutations)
- **Range**: 1.0 to 2.0 (one octave above root)

## 53-TET System
- 53 equal divisions of the octave
- Step size = 2^(1/53) ≈ 1.01316 (≈ 22.64 cents)
- Closely approximates just intonation intervals
- Superior to 12-TET for harmonic accuracy

## Goals
1. Define all 53-TET chord positions in EigenSpace
2. Find local minima (harmonic nodes) in the dissonance map  
3. Compare 53-TET chords with 12-TET positions
4. Create reference data for microtonal augmentation pipeline

In [110]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.ndimage import minimum_filter, maximum_filter
import os
from typing import List, Tuple, Dict, Optional
import json

# ============================================================================
# 53-TET FUNDAMENTAL FUNCTIONS
# ============================================================================

def get_53tet_ratio(steps: int) -> float:
    """
    Convert 53-TET steps to frequency ratio.
    
    53-TET divides the octave (2:1) into 53 equal parts.
    Each step = 2^(1/53) ≈ 1.01316 ratio (≈ 22.64 cents)
    
    Args:
        steps: Number of 53-TET steps from root (0-53)
    
    Returns:
        Frequency ratio relative to root
    """
    return 2.0 ** (steps / 53.0)


def get_12tet_ratio(semitones: int) -> float:
    """
    Convert 12-TET semitones to frequency ratio.
    
    Args:
        semitones: Number of semitones from root (0-12)
    
    Returns:
        Frequency ratio relative to root
    """
    return 2.0 ** (semitones / 12.0)


# Helper shorthand for ratio conversion
r = get_53tet_ratio
r12 = get_12tet_ratio

print("53-TET step examples:")
print(f"  Major third (18 steps):  {r(18):.6f}  (vs JI 5/4 = {5/4:.6f})")
print(f"  Perfect fifth (31 steps): {r(31):.6f}  (vs JI 3/2 = {3/2:.6f})")
print(f"  Major seventh (49 steps): {r(49):.6f}  (vs JI 15/8 = {15/8:.6f})")

53-TET step examples:
  Major third (18 steps):  1.265426  (vs JI 5/4 = 1.250000)
  Perfect fifth (31 steps): 1.499941  (vs JI 3/2 = 1.500000)
  Major seventh (49 steps): 1.898064  (vs JI 15/8 = 1.875000)


In [111]:
# ============================================================================
# 53-TET INTERVAL DEFINITIONS
# ============================================================================
# These match exactly the JavaScript definitions from Harmonic_Eigenspace.js

# THIRDS (from root) - 53-TET step values
THIRDS_53 = {
    'sm': 11,   # super-minor (wolf third)
    'vm': 12,   # sub-minor
    'm':  13,   # minor third (300 cents in 12-TET → 13.25 steps)
    '^m': 14,   # super-minor / neutral low
    'n':  15,   # neutral third
    'N':  16,   # neutral third high
    'vM': 17,   # sub-major
    'M':  18,   # major third (400 cents → 17.67 steps, rounds to 18)
    '^M': 19,   # super-major
    'SM': 20,   # wide major (septimal)
}

# FIFTHS (from root) - 53-TET step values  
FIFTHS_53 = {
    'subdim': 29,  # sub-diminished
    'dim':    30,  # diminished fifth
    'vP':     30,  # sub-perfect (same as dim)
    'P':      31,  # perfect fifth (700 cents → 30.92 steps)
    '^P':     32,  # super-perfect
    'aug':    32,  # augmented fifth (same as ^P)
    'upaug':  33,  # up-augmented
}

# SEVENTHS (from root) - 53-TET step values
SEVENTHS_53 = {
    'sm': 42,  # super-diminished 7th
    'vm': 43,  # sub-minor 7th
    'm':  44,  # minor seventh (1000 cents → 44.17 steps)
    '^m': 45,  # super-minor 7th
    'n':  46,  # neutral 7th
    'N':  47,  # neutral 7th high
    'vM': 48,  # sub-major 7th
    'M':  49,  # major seventh (1100 cents → 48.58 steps)
    '^M': 50,  # super-major 7th
    'SM': 51,  # wide major 7th
}

print("53-TET Interval Mappings:")
print(f"\nThirds (steps → ratio → cents):")
for name, steps in THIRDS_53.items():
    ratio = r(steps)
    cents = 1200 * np.log2(ratio)
    print(f"  {name:>3}: {steps:2d} steps → {ratio:.5f} → {cents:.1f}¢")

print(f"\nFifths:")
for name, steps in FIFTHS_53.items():
    ratio = r(steps)
    cents = 1200 * np.log2(ratio)
    print(f"  {name:>6}: {steps:2d} steps → {ratio:.5f} → {cents:.1f}¢")

print(f"\nSevenths:")
for name, steps in SEVENTHS_53.items():
    ratio = r(steps)
    cents = 1200 * np.log2(ratio)
    print(f"  {name:>3}: {steps:2d} steps → {ratio:.5f} → {cents:.1f}¢")

53-TET Interval Mappings:

Thirds (steps → ratio → cents):
   sm: 11 steps → 1.15472 → 249.1¢
   vm: 12 steps → 1.16992 → 271.7¢
    m: 13 steps → 1.18533 → 294.3¢
   ^m: 14 steps → 1.20093 → 317.0¢
    n: 15 steps → 1.21674 → 339.6¢
    N: 16 steps → 1.23276 → 362.3¢
   vM: 17 steps → 1.24898 → 384.9¢
    M: 18 steps → 1.26543 → 407.5¢
   ^M: 19 steps → 1.28208 → 430.2¢
   SM: 20 steps → 1.29896 → 452.8¢

Fifths:
  subdim: 29 steps → 1.46122 → 656.6¢
     dim: 30 steps → 1.48045 → 679.2¢
      vP: 30 steps → 1.48045 → 679.2¢
       P: 31 steps → 1.49994 → 701.9¢
      ^P: 32 steps → 1.51969 → 724.5¢
     aug: 32 steps → 1.51969 → 724.5¢
   upaug: 33 steps → 1.53969 → 747.2¢

Sevenths:
   sm: 42 steps → 1.73202 → 950.9¢
   vm: 43 steps → 1.75482 → 973.6¢
    m: 44 steps → 1.77792 → 996.2¢
   ^m: 45 steps → 1.80132 → 1018.9¢
    n: 46 steps → 1.82504 → 1041.5¢
    N: 47 steps → 1.84906 → 1064.2¢
   vM: 48 steps → 1.87340 → 1086.8¢
    M: 49 steps → 1.89806 → 1109.4¢
   ^M: 50 steps → 1.

In [112]:
# ============================================================================
# 53-TET CHORD DEFINITIONS - EXACT PORT FROM Harmonic_Eigenspace.js
# ============================================================================

def get_53tet_chord_positions() -> List[Tuple[str, float, float, float]]:
    """
    Get all 53-TET chord positions in EigenSpace (α, β, γ) coordinates.
    
    DIRECT PORT from get53TETChordPositions() in Harmonic_Eigenspace.js
    
    Structure: Each chord is [name, r(third), r(fifth), r(seventh)]
    - Organized by third type (SM=20 down to sm=11)
    - Each third has 10 seventh variants
    - Most use perfect fifth (31), half-dim uses 26
    
    Returns:
        List of (name, α, β, γ) tuples
    """
    chords = []
    
    # ===== SUPER-MAJOR THIRD (SM=20) COMBINATIONS =====
    chords.extend([
        ("SMSM7",   r(20), r(31), r(51)),
        ("SM^M7",   r(20), r(31), r(50)),
        ("SMmaj7",  r(20), r(31), r(49)),
        ("SMvM7",   r(20), r(31), r(48)),
        ("SMN7",    r(20), r(31), r(47)),
        ("SMn7",    r(20), r(31), r(46)),
        ("SM^m7",   r(20), r(31), r(45)),
        ("SMm7",    r(20), r(31), r(44)),
        ("SMvm7",   r(20), r(31), r(43)),
        ("SMsm7",   r(20), r(31), r(42)),
    ])
    
    # ===== UP-MAJOR THIRD (^M=19) COMBINATIONS =====
    chords.extend([
        ("^MSM7",   r(19), r(31), r(51)),
        ("^M^M7",   r(19), r(31), r(50)),
        ("^Mmaj7",  r(19), r(31), r(49)),
        ("^MvM7",   r(19), r(31), r(48)),
        ("^MN7",    r(19), r(31), r(47)),
        ("^Mn7",    r(19), r(31), r(46)),
        ("^M^m7",   r(19), r(31), r(45)),
        ("^Mm7",    r(19), r(31), r(44)),
        ("^Mvm7",   r(19), r(31), r(43)),
        ("^Msm7",   r(19), r(31), r(42)),
    ])
    
    # ===== MAJOR THIRD (M=18) COMBINATIONS =====
    chords.extend([
        ("MSM7",    r(18), r(31), r(51)),
        ("M^M7",    r(18), r(31), r(50)),
        ("maj7",    r(18), r(31), r(49)),
        ("MvM7",    r(18), r(31), r(48)),
        ("MN7",     r(18), r(31), r(47)),
        ("Mn7",     r(18), r(31), r(46)),
        ("M^m7",    r(18), r(31), r(45)),
        ("Mm7",     r(18), r(31), r(44)),
        ("Mvm7",    r(18), r(31), r(43)),
        ("Msm7",    r(18), r(31), r(42)),
    ])
    
    # ===== DOWN-MAJOR THIRD (vM=17) COMBINATIONS =====
    chords.extend([
        ("vMSM7",   r(17), r(31), r(51)),
        ("vM^M7",   r(17), r(31), r(50)),
        ("vMmaj7",  r(17), r(31), r(49)),
        ("vMvM7",   r(17), r(31), r(48)),
        ("vMN7",    r(17), r(31), r(47)),
        ("vMn7",    r(17), r(31), r(46)),
        ("vM^m7",   r(17), r(31), r(45)),
        ("vMm7",    r(17), r(31), r(44)),
        ("vMvm7",   r(17), r(31), r(43)),
        ("vMsm7",   r(17), r(31), r(42)),
    ])
    
    # ===== NEUTRAL-MAJOR THIRD (N=16) COMBINATIONS =====
    chords.extend([
        ("NSM7",    r(16), r(31), r(51)),
        ("N^M7",    r(16), r(31), r(50)),
        ("Nmaj7",   r(16), r(31), r(49)),
        ("NvM7",    r(16), r(31), r(48)),
        ("NN7",     r(16), r(31), r(47)),
        ("Nn7",     r(16), r(31), r(46)),
        ("N^m7",    r(16), r(31), r(45)),
        ("Nm7",     r(16), r(31), r(44)),
        ("Nvm7",    r(16), r(31), r(43)),
        ("Nsm7",    r(16), r(31), r(42)),
    ])
    
    # ===== NEUTRAL-MINOR THIRD (n=15) COMBINATIONS =====
    chords.extend([
        ("nSM7",    r(15), r(31), r(51)),
        ("n^M7",    r(15), r(31), r(50)),
        ("nmaj7",   r(15), r(31), r(49)),
        ("nvM7",    r(15), r(31), r(48)),
        ("nN7",     r(15), r(31), r(47)),
        ("nn7",     r(15), r(31), r(46)),
        ("n^m7",    r(15), r(31), r(45)),
        ("nm7",     r(15), r(31), r(44)),
        ("nvm7",    r(15), r(31), r(43)),
        ("nsm7",    r(15), r(31), r(42)),
    ])
    
    # ===== UP-MINOR THIRD (^m=14) COMBINATIONS =====
    chords.extend([
        ("^mSM7",   r(14), r(31), r(51)),
        ("^m^M7",   r(14), r(31), r(50)),
        ("^mmaj7",  r(14), r(31), r(49)),
        ("^mvM7",   r(14), r(31), r(48)),
        ("^mN7",    r(14), r(31), r(47)),
        ("^mn7",    r(14), r(31), r(46)),
        ("^m^m7",   r(14), r(31), r(45)),
        ("^mm7",    r(14), r(31), r(44)),
        ("^mvm7",   r(14), r(31), r(43)),
        ("^msm7",   r(14), r(31), r(42)),
    ])
    
    # ===== MINOR THIRD (m=13) COMBINATIONS =====
    chords.extend([
        ("mSM7",    r(13), r(31), r(51)),
        ("m^M7",    r(13), r(31), r(50)),
        ("mmaj7",   r(13), r(31), r(49)),
        ("mvM7",    r(13), r(31), r(48)),
        ("mN7",     r(13), r(31), r(47)),
        ("mn7",     r(13), r(31), r(46)),
        ("m^m7",    r(13), r(31), r(45)),
        ("m7",      r(13), r(31), r(44)),
        ("mvm7",    r(13), r(31), r(43)),
        ("msm7",    r(13), r(31), r(42)),
    ])
    
    # ===== DOWN-MINOR THIRD (vm=12) COMBINATIONS =====
    chords.extend([
        ("vmSM7",   r(12), r(31), r(51)),
        ("vm^M7",   r(12), r(31), r(50)),
        ("vmmaj7",  r(12), r(31), r(49)),
        ("vmvM7",   r(12), r(31), r(48)),
        ("vmN7",    r(12), r(31), r(47)),
        ("vmn7",    r(12), r(31), r(46)),
        ("vm^m7",   r(12), r(31), r(45)),
        ("vm7",     r(12), r(31), r(44)),
        ("vmvm7",   r(12), r(31), r(43)),
        ("vmsm7",   r(12), r(31), r(42)),
    ])
    
    # ===== SUB-MINOR THIRD (sm=11) COMBINATIONS =====
    chords.extend([
        ("smSM7",   r(11), r(31), r(51)),
        ("sm^M7",   r(11), r(31), r(50)),
        ("smmaj7",  r(11), r(31), r(49)),
        ("smvM7",   r(11), r(31), r(48)),
        ("smN7",    r(11), r(31), r(47)),
        ("smn7",    r(11), r(31), r(46)),
        ("sm^m7",   r(11), r(31), r(45)),
        ("sm7",     r(11), r(31), r(44)),
        ("smvm7",   r(11), r(31), r(43)),
        ("smsm7",   r(11), r(31), r(42)),
    ])
    
    # ===== HALF-DIMINISHED UP-MINOR (^m=14, fifth=26) =====
    chords.extend([
        ("øSM7",    r(14), r(26), r(51)),
        ("ø^M7",    r(14), r(26), r(50)),
        ("ømaj7",   r(14), r(26), r(49)),
        ("øvM7",    r(14), r(26), r(48)),
        ("øN7",     r(14), r(26), r(47)),
        ("øn7",     r(14), r(26), r(46)),
        ("ø^m7",    r(14), r(26), r(45)),
        ("ø7",      r(14), r(26), r(44)),
        ("øvm7",    r(14), r(26), r(43)),
        ("øsm7",    r(14), r(26), r(42)),
    ])
    
    # ===== HALF-DIMINISHED MINOR (m=13, fifth=26) =====
    chords.extend([
        ("øS7",     r(13), r(26), r(51)),
        ("ø^M7-m",  r(13), r(26), r(50)),
        ("ømaj7-m", r(13), r(26), r(49)),
        ("øvM7-m",  r(13), r(26), r(48)),
        ("øNM7",    r(13), r(26), r(47)),
        ("øn7-m",   r(13), r(26), r(46)),
        ("øv7",     r(13), r(26), r(45)),
        ("ø7-m",    r(13), r(26), r(44)),
        ("øvm7-m",  r(13), r(26), r(43)),
        ("øsm7-m",  r(13), r(26), r(42)),
    ])
    
    # ===== HALF-DIMINISHED DOWN-MINOR (vm=12, fifth=26) =====
    chords.extend([
        ("vøS7",    r(12), r(26), r(51)),
        ("vø^M7",   r(12), r(26), r(50)),
        ("vømaj7",  r(12), r(26), r(49)),
        ("vøvM7",   r(12), r(26), r(48)),
        ("vøN7",    r(12), r(26), r(47)),
        ("vøn7",    r(12), r(26), r(46)),
        ("vø^m7",   r(12), r(26), r(45)),
        ("vø7",     r(12), r(26), r(44)),
        ("vøvm7",   r(12), r(26), r(43)),
        ("vøsm7",   r(12), r(26), r(42)),
    ])
    
    # ===== HALF-DIMINISHED SUB-MINOR (sm=11, fifth=26) =====
    chords.extend([
        ("søS7",    r(11), r(26), r(51)),
        ("sø^M7",   r(11), r(26), r(50)),
        ("sømaj7",  r(11), r(26), r(49)),
        ("søvM7",   r(11), r(26), r(48)),
        ("søN7",    r(11), r(26), r(47)),
        ("søn7",    r(11), r(26), r(46)),
        ("sø^m7",   r(11), r(26), r(45)),
        ("sø7",     r(11), r(26), r(44)),
        ("søvm7",   r(11), r(26), r(43)),
        ("søsm7",   r(11), r(26), r(42)),
    ])
    
    # ===== AUGMENTED (M=18, fifth=35) =====
    chords.extend([
        ("M+S7",    r(18), r(35), r(51)),
        ("M+^M7",   r(18), r(35), r(50)),
        ("M+maj7",  r(18), r(35), r(49)),
        ("M+vM7",   r(18), r(35), r(48)),
        ("M+NM7",   r(18), r(35), r(47)),
        ("M+N7",    r(18), r(35), r(46)),
        ("M+n7",    r(18), r(35), r(45)),
        ("M+m7",    r(18), r(35), r(44)),
        ("M+vm7",   r(18), r(35), r(43)),
        ("M+sm7",   r(18), r(35), r(42)),
    ])
    
    # ===== SUSPENDED =====
    chords.extend([
        ("7sus4",   r(22), r(31), r(44)),
        ("sus2",    r(9),  r(22), r(31)),
    ])
    
    return chords


# Generate the chord list
chords_53tet = get_53tet_chord_positions()
print(f"Total 53-TET chords defined: {len(chords_53tet)}")
print("\nFirst 15 chords (Super-Major third family):")
for name, alpha, beta, gamma in chords_53tet[:15]:
    print(f"  {name:12s}: α={alpha:.5f}, β={beta:.5f}, γ={gamma:.5f}")
print("\n...")
print("\nHalf-diminished family (ø7 variants):")
for name, alpha, beta, gamma in chords_53tet[100:110]:
    print(f"  {name:12s}: α={alpha:.5f}, β={beta:.5f}, γ={gamma:.5f}")

Total 53-TET chords defined: 152

First 15 chords (Super-Major third family):
  SMSM7       : α=1.29896, β=1.49994, γ=1.94837
  SM^M7       : α=1.29896, β=1.49994, γ=1.92305
  SMmaj7      : α=1.29896, β=1.49994, γ=1.89806
  SMvM7       : α=1.29896, β=1.49994, γ=1.87340
  SMN7        : α=1.29896, β=1.49994, γ=1.84906
  SMn7        : α=1.29896, β=1.49994, γ=1.82504
  SM^m7       : α=1.29896, β=1.49994, γ=1.80132
  SMm7        : α=1.29896, β=1.49994, γ=1.77792
  SMvm7       : α=1.29896, β=1.49994, γ=1.75482
  SMsm7       : α=1.29896, β=1.49994, γ=1.73202
  ^MSM7       : α=1.28208, β=1.49994, γ=1.94837
  ^M^M7       : α=1.28208, β=1.49994, γ=1.92305
  ^Mmaj7      : α=1.28208, β=1.49994, γ=1.89806
  ^MvM7       : α=1.28208, β=1.49994, γ=1.87340
  ^MN7        : α=1.28208, β=1.49994, γ=1.84906

...

Half-diminished family (ø7 variants):
  øSM7        : α=1.20093, β=1.40500, γ=1.94837
  ø^M7        : α=1.20093, β=1.40500, γ=1.92305
  ømaj7       : α=1.20093, β=1.40500, γ=1.89806
  øvM7        

In [113]:
# ============================================================================
# 12-TET CHORD DEFINITIONS - FROM Harmonic_Eigenspace.js get12TETChordPositions()
# ============================================================================

def get_12tet_chord_positions() -> List[Tuple[str, float, float, float]]:
    """
    Get standard 12-TET chord positions - direct port from JS.
    
    Returns:
        List of (name, α, β, γ) tuples
    """
    return [
        ("maj7",    r12(4),  r12(7),  r12(11)),
        ("min7",    r12(3),  r12(7),  r12(10)),
        ("dom7",    r12(4),  r12(7),  r12(10)),
        ("mM7",     r12(3),  r12(7),  r12(11)),
        ("ø7",      r12(3),  r12(6),  r12(10)),
        ("dim7",    r12(3),  r12(6),  r12(9)),
        ("aug7",    r12(4),  r12(8),  r12(10)),
        ("augM7",   r12(4),  r12(8),  r12(11)),
        ("7sus4",   r12(5),  r12(7),  r12(10)),
        ("sus2",    r12(2),  r12(5),  r12(7)),
    ]


chords_12tet = get_12tet_chord_positions()
print(f"Total 12-TET reference chords: {len(chords_12tet)}")
print("\n12-TET chord positions:")
for name, alpha, beta, gamma in chords_12tet:
    print(f"  {name:10s}: α={alpha:.5f}, β={beta:.5f}, γ={gamma:.5f}")

Total 12-TET reference chords: 10

12-TET chord positions:
  maj7      : α=1.25992, β=1.49831, γ=1.88775
  min7      : α=1.18921, β=1.49831, γ=1.78180
  dom7      : α=1.25992, β=1.49831, γ=1.78180
  mM7       : α=1.18921, β=1.49831, γ=1.88775
  ø7        : α=1.18921, β=1.41421, γ=1.78180
  dim7      : α=1.18921, β=1.41421, γ=1.68179
  aug7      : α=1.25992, β=1.58740, γ=1.78180
  augM7     : α=1.25992, β=1.58740, γ=1.88775
  7sus4     : α=1.33484, β=1.49831, γ=1.78180
  sus2      : α=1.12246, β=1.33484, γ=1.49831


## Load Pre-computed Dissonance Map

The EigenSpace dissonance map was pre-computed at 220 Hz with:
- 150³ = 3,375,000 points  
- α, β, γ range: 1.0 to 2.0
- Plomp-Levelt dissonance model with 6 harmonics

In [114]:
# ============================================================================
# LOAD PRE-COMPUTED DISSONANCE MAP
# ============================================================================

def load_precomputed_dissonance_map(
    base_freq: int = 220,
    n_points: int = 150,
    r_low: float = 1.0,
    r_high: float = 2.0,
    dataset_path: str = "../dataset/EigenSpace_Data"
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Load pre-computed dissonance map from binary chunk files.
    
    Args:
        base_freq: Base frequency the map was computed at (Hz)
        n_points: Resolution (n³ grid points)
        r_low, r_high: Ratio range
        dataset_path: Path to EigenSpace_Data folder
        
    Returns:
        (alpha_range, beta_range, gamma_range, dissonance_3d)
    """
    print(f"Loading {n_points}³ pre-computed dataset from {dataset_path}...")
    
    # Find all chunk files
    chunk_files = sorted([
        f for f in os.listdir(dataset_path)
        if f.startswith(f"harmonic-{base_freq}Hz-{n_points}nodes-chunk")
    ])
    
    if not chunk_files:
        raise FileNotFoundError(
            f"No dataset chunks found for {base_freq}Hz, {n_points} nodes"
        )
    
    # Load all chunks
    all_data = []
    for chunk_file in chunk_files:
        chunk_path = os.path.join(dataset_path, chunk_file)
        chunk_data = np.fromfile(chunk_path, dtype=np.float32)
        all_data.append(chunk_data)
        print(f"  ✓ {chunk_file}: {len(chunk_data):,} values")
    
    # Concatenate and reshape
    flat_data = np.concatenate(all_data)
    dissonance_3d = flat_data.reshape((n_points, n_points, n_points))
    
    # Recreate coordinate ranges
    alpha_range = np.linspace(r_low, r_high, n_points)
    beta_range = np.linspace(r_low, r_high, n_points)
    gamma_range = np.linspace(r_low, r_high, n_points)
    
    print(f"\n✓ Loaded shape: {dissonance_3d.shape}")
    print(f"✓ Dissonance range: {dissonance_3d.min():.4f} to {dissonance_3d.max():.4f}")
    
    return alpha_range, beta_range, gamma_range, dissonance_3d


# Load the dataset
alpha_range, beta_range, gamma_range, dissonance_3d = load_precomputed_dissonance_map(
    base_freq=220,
    n_points=150
)

Loading 150³ pre-computed dataset from ../dataset/EigenSpace_Data...
  ✓ harmonic-220Hz-150nodes-chunk001.bin: 1,000,000 values
  ✓ harmonic-220Hz-150nodes-chunk002.bin: 1,000,000 values
  ✓ harmonic-220Hz-150nodes-chunk003.bin: 1,000,000 values
  ✓ harmonic-220Hz-150nodes-chunk004.bin: 375,000 values

✓ Loaded shape: (150, 150, 150)
✓ Dissonance range: 0.6725 to 30.6237


In [115]:
# ============================================================================
# DISSONANCE COMPUTATION — Direct Plomp-Levelt (exact, per-root frequency)
# ============================================================================
#
# The Plomp-Levelt critical bandwidth parameter S = Dstar / (S1*Fmin + S2)
# depends on ABSOLUTE frequency. The dissonance landscape varies with the
# root pitch — low-register chords have wider critical bands = more dissonance.
#
# EigenSpace (α, β, γ) are ratio vectors from the root. The 4th coordinate D
# must be computed from the ACTUAL root frequency of each chord, not a fixed
# reference. Each chord's root is the origin of its own dissonance vector.

def dissmeasure(fvec, amp, model="min"):
    """
    Plomp-Levelt dissonance measure — exact port from Harmonic_Eigenspace.js.
    
    Args:
        fvec: List of frequencies (Hz)
        amp: List of amplitudes
        model: "min" uses min(a_i, a_j) for pair weighting
        
    Returns:
        Total dissonance (float)
    """
    sorted_pairs = sorted(zip(fvec, amp), key=lambda x: x[0])
    fr = [p[0] for p in sorted_pairs]
    am = [p[1] for p in sorted_pairs]
    
    Dstar = 0.24
    S1, S2 = 0.0207, 18.96
    C1, C2 = 5.0, -5.0
    A1, A2 = -3.51, -5.75
    
    total = 0.0
    for i in range(len(fr)):
        for j in range(i + 1, len(fr)):
            Fmin = fr[i]
            S = Dstar / (S1 * Fmin + S2)
            Fdif = fr[j] - fr[i]
            a = min(am[i], am[j])
            SFdif = S * Fdif
            total += a * (C1 * np.exp(A1 * SFdif) + C2 * np.exp(A2 * SFdif))
    return total


def step53_to_hz(step_53: int) -> float:
    """
    Convert an absolute 53-TET step number to frequency in Hz.
    
    Mapping: MIDI note 69 (A4) = 440 Hz.
    MIDI note n → 53-TET step = round(n * 53/12).
    Inverse: step → freq = 440 * 2^((step/53) - (69/12))
    """
    return 440.0 * (2.0 ** (step_53 / 53.0 - 69.0 / 12.0))


def compute_dissonance(alpha: float, beta: float, gamma: float,
                       base_freq: float = 220.0, num_harmonics: int = 6) -> float:
    """
    Compute Plomp-Levelt dissonance for a chord at a given root frequency.
    
    The (α, β, γ, D) tuple IS the EigenSpace 4D coordinate.
    D depends on the root pitch because critical bandwidth is frequency-dependent.
    
    Args:
        alpha: Frequency ratio of 2nd note to root
        beta:  Frequency ratio of 3rd note to root
        gamma: Frequency ratio of 4th note to root
        base_freq: ROOT frequency in Hz — must be the actual pitch of the chord root
        num_harmonics: Number of harmonics per voice (6 = standard)
        
    Returns:
        Dissonance D (float). Lower = more consonant.
    """
    freqs = []
    amps = []
    for h in range(1, num_harmonics + 1):
        freqs.extend([
            base_freq * h,
            base_freq * h * alpha,
            base_freq * h * beta,
            base_freq * h * gamma,
        ])
        amps.extend([1.0, 1.0, 1.0, 1.0])
    return dissmeasure(freqs, amps)


# --- Map lookup (kept for 3D visualization only) ---

def get_dissonance_at_point(
    alpha: float, beta: float, gamma: float,
    alpha_range: np.ndarray, beta_range: np.ndarray, gamma_range: np.ndarray,
    dissonance_3d: np.ndarray
) -> Optional[float]:
    """
    Get dissonance from pre-computed 150³ map (trilinear interpolation).
    ⚠ USE compute_dissonance() with actual root Hz for accurate queries.
    """
    if (alpha < alpha_range[0] or alpha > alpha_range[-1] or
        beta < beta_range[0] or beta > beta_range[-1] or
        gamma < gamma_range[0] or gamma > gamma_range[-1]):
        return None
    
    def find_bracket(val, arr):
        idx = np.searchsorted(arr, val) - 1
        idx = max(0, min(idx, len(arr) - 2))
        t = (val - arr[idx]) / (arr[idx + 1] - arr[idx])
        return idx, t
    
    i, ti = find_bracket(alpha, alpha_range)
    j, tj = find_bracket(beta, beta_range)
    k, tk = find_bracket(gamma, gamma_range)
    
    c000 = dissonance_3d[i, j, k]
    c100 = dissonance_3d[i+1, j, k]
    c010 = dissonance_3d[i, j+1, k]
    c110 = dissonance_3d[i+1, j+1, k]
    c001 = dissonance_3d[i, j, k+1]
    c101 = dissonance_3d[i+1, j, k+1]
    c011 = dissonance_3d[i, j+1, k+1]
    c111 = dissonance_3d[i+1, j+1, k+1]
    
    c00 = c000 * (1 - ti) + c100 * ti
    c10 = c010 * (1 - ti) + c110 * ti
    c01 = c001 * (1 - ti) + c101 * ti
    c11 = c011 * (1 - ti) + c111 * ti
    c0 = c00 * (1 - tj) + c10 * tj
    c1 = c01 * (1 - tj) + c11 * tj
    return c0 * (1 - tk) + c1 * tk


# --- Demonstrate frequency dependence ---
print("Plomp-Levelt frequency dependence: same ratios, different root pitch")
print("=" * 80)
print(f"  {'Chord':<14} {'Root Hz':>8}  {'α':>7} {'β':>7} {'γ':>7}  {'D':>8}")
print("-" * 80)
for label, a, b, g in [
    ("maj7 (53-TET)",  r(18), r(31), r(49)),
    ("m7   (53-TET)",  r(13), r(31), r(44)),
    ("dom7 (53-TET)",  r(18), r(31), r(44)),
]:
    for root_hz in [65, 130, 220, 261, 440, 523]:
        d = compute_dissonance(a, b, g, base_freq=root_hz)
        note_name = {65:"C2", 130:"C3", 220:"A3", 261:"C4", 440:"A4", 523:"C5"}[root_hz]
        print(f"  {label:<14} {note_name:>4} {root_hz:>3}  {a:>7.4f} {b:>7.4f} {g:>7.4f}  {d:>8.4f}")
    print()

print("✓ Same interval ratios, DIFFERENT dissonance at different registers")
print("  → Each chord must be computed from its actual root frequency")

Plomp-Levelt frequency dependence: same ratios, different root pitch
  Chord           Root Hz        α       β       γ         D
--------------------------------------------------------------------------------
  maj7 (53-TET)    C2  65   1.2654  1.4999  1.8981   38.3907
  maj7 (53-TET)    C3 130   1.2654  1.4999  1.8981   20.2860
  maj7 (53-TET)    A3 220   1.2654  1.4999  1.8981   13.5842
  maj7 (53-TET)    C4 261   1.2654  1.4999  1.8981   12.2002
  maj7 (53-TET)    A4 440   1.2654  1.4999  1.8981    9.4351
  maj7 (53-TET)    C5 523   1.2654  1.4999  1.8981    8.8537

  m7   (53-TET)    C2  65   1.1853  1.4999  1.7779   40.8080
  m7   (53-TET)    C3 130   1.1853  1.4999  1.7779   21.1142
  m7   (53-TET)    A3 220   1.1853  1.4999  1.7779   13.2874
  m7   (53-TET)    C4 261   1.1853  1.4999  1.7779   11.6115
  m7   (53-TET)    A4 440   1.1853  1.4999  1.7779    8.2334
  m7   (53-TET)    C5 523   1.1853  1.4999  1.7779    7.5357

  dom7 (53-TET)    C2  65   1.2654  1.4999  1.7779   42

In [116]:
# ============================================================================
# DIAGNOSTIC: Investigate axis ordering bug
# ============================================================================
import itertools

print("=" * 70)
print("DIAGNOSTIC: Axis ordering and data integrity check")
print("=" * 70)

# 1. Check raw data at boundaries
print("\n--- 1. Corner values (raw array access) ---")
print(f"dissonance_3d[0,0,0]       = {dissonance_3d[0,0,0]:.4f}   (all axes=1.0, unison)")
print(f"dissonance_3d[149,149,149] = {dissonance_3d[149,149,149]:.4f}   (all axes=2.0, octave)")
print(f"dissonance_3d[0,0,149]     = {dissonance_3d[0,0,149]:.4f}")
print(f"dissonance_3d[149,0,0]     = {dissonance_3d[149,0,0]:.4f}")
print(f"dissonance_3d[0,149,0]     = {dissonance_3d[0,149,0]:.4f}")
print(f"\nNaN count: {np.isnan(dissonance_3d).sum():,} out of {dissonance_3d.size:,}")
print(f"Data range: {dissonance_3d.min():.4f} to {dissonance_3d.max():.4f}")

# 2. Check symmetry (the dissonance function IS symmetric in α,β,γ)
print("\n--- 2. Symmetry check ---")
idx_a, idx_b, idx_c = 20, 80, 120
perms_check = list(itertools.permutations([idx_a, idx_b, idx_c]))
for p in perms_check:
    print(f"  d3d[{p[0]:3d},{p[1]:3d},{p[2]:3d}] = {dissonance_3d[p[0],p[1],p[2]]:.6f}")

# 3. Compute dissonance directly for known chords and compare
print("\n--- 3. Direct computation vs map lookup for known chords ---")

def dissmeasure_py(fvec, amp, model="min"):
    sorted_pairs = sorted(zip(fvec, amp), key=lambda x: x[0])
    fr_sorted = [p[0] for p in sorted_pairs]
    am_sorted = [p[1] for p in sorted_pairs]
    Dstar, S1, S2 = 0.24, 0.0207, 18.96
    C1, C2 = 5, -5
    A1, A2 = -3.51, -5.75
    total = 0
    for i in range(len(fr_sorted)):
        for j in range(i+1, len(fr_sorted)):
            Fmin = fr_sorted[i]
            S = Dstar / (S1 * Fmin + S2)
            Fdif = fr_sorted[j] - fr_sorted[i]
            a = min(am_sorted[i], am_sorted[j])
            SFdif = S * Fdif
            total += a * (C1 * np.exp(A1 * SFdif) + C2 * np.exp(A2 * SFdif))
    return total

def compute_chord_dissonance(alpha, beta, gamma, base_freq=220, num_harmonics=6):
    freqs = []
    amps = []
    for h in range(1, num_harmonics + 1):
        freqs.extend([base_freq * h, base_freq * h * alpha,
                       base_freq * h * beta, base_freq * h * gamma])
        amps.extend([1.0, 1.0, 1.0, 1.0])
    return dissmeasure_py(freqs, amps)

test_chords = [
    ("12-TET maj7",     r12(4), r12(7), r12(11)),
    ("12-TET min7",     r12(3), r12(7), r12(10)),
    ("12-TET dom7",     r12(4), r12(7), r12(10)),
    ("53-TET maj7",     r(18),  r(31),  r(49)),
    ("53-TET m7",       r(13),  r(31),  r(44)),
    ("JI maj triad+7",  5/4,    3/2,    15/8),
    ("JI min triad+m7", 6/5,    3/2,    9/5),
    ("unison",          1.001,  1.001,  1.001),
    ("octave triple",   1.999,  1.999,  1.999),
    ("perf5 trio",      1.5,    1.5,    1.5),
]

print(f"\n{'Chord':<22} {'α':>7} {'β':>7} {'γ':>7} {'Map val':>9} {'Direct':>9} {'Match?':>7}")
print("-" * 75)
for name, a, b, g in test_chords:
    map_val = get_dissonance_at_point(a, b, g, alpha_range, beta_range, gamma_range, dissonance_3d)
    direct_val = compute_chord_dissonance(a, b, g)
    match = "✓" if map_val and abs(map_val - direct_val) < 0.5 else "✗"
    map_str = f"{map_val:.4f}" if map_val else "None"
    print(f"{name:<22} {a:>7.4f} {b:>7.4f} {g:>7.4f} {map_str:>9} {direct_val:>9.4f} {match:>7}")

# 4. What num_harmonics was the data computed with?
# Test by computing at a known grid point with different harmonic counts
print("\n--- 4. Detecting computation parameters ---")
grid_idx = 75  # middle of grid
grid_val = alpha_range[grid_idx]
grid_diss = dissonance_3d[grid_idx, grid_idx, grid_idx]
print(f"Grid point [{grid_idx},{grid_idx},{grid_idx}] = ({grid_val:.4f}, {grid_val:.4f}, {grid_val:.4f}), stored value = {grid_diss:.6f}")

for n_harm in [4, 5, 6, 7, 8]:
    for base_f in [220, 500]:
        computed = compute_chord_dissonance(grid_val, grid_val, grid_val, base_freq=base_f, num_harmonics=n_harm)
        match = "← MATCH" if abs(computed - grid_diss) < 0.05 else ""
        print(f"  {base_f}Hz, {n_harm} harmonics: {computed:.6f} {match}")

# 5. Try ALL 6 axis permutations — which gives best match?
print("\n--- 5. Test ALL axis permutations ---")
perms = list(itertools.permutations([0, 1, 2]))
perm_labels = ["α,β,γ", "α,γ,β", "β,α,γ", "β,γ,α", "γ,α,β", "γ,β,α"]

for perm, label in zip(perms, perm_labels):
    total_err = 0
    n_valid = 0
    for name, a, b, g in test_chords[:7]:
        coords = [a, b, g]
        # Map coordinates to array indices via permutation
        # perm[0]=which coordinate maps to axis0, etc.
        i0 = np.searchsorted(alpha_range, coords[perm[0]]) - 1
        i1 = np.searchsorted(beta_range,  coords[perm[1]]) - 1
        i2 = np.searchsorted(gamma_range, coords[perm[2]]) - 1
        i0 = max(0, min(i0, 148))
        i1 = max(0, min(i1, 148))
        i2 = max(0, min(i2, 148))
        
        map_val = dissonance_3d[i0, i1, i2]
        direct_val = compute_chord_dissonance(a, b, g)
        if not np.isnan(map_val):
            total_err += abs(map_val - direct_val)
            n_valid += 1
    
    avg_err = total_err / n_valid if n_valid > 0 else 999
    print(f"  axes=[{label}]: avg |error| = {avg_err:.4f}")


DIAGNOSTIC: Axis ordering and data integrity check

--- 1. Corner values (raw array access) ---
dissonance_3d[0,0,0]       = 1.8189   (all axes=1.0, unison)
dissonance_3d[149,149,149] = 0.6725   (all axes=2.0, octave)
dissonance_3d[0,0,149]     = 1.3933
dissonance_3d[149,0,0]     = 1.3933
dissonance_3d[0,149,0]     = 1.3933

NaN count: 0 out of 3,375,000
Data range: 0.6725 to 30.6237

--- 2. Symmetry check ---
  d3d[ 20, 80,120] = 17.096001
  d3d[ 20,120, 80] = 17.096001
  d3d[ 80, 20,120] = 17.096001
  d3d[ 80,120, 20] = 17.096001
  d3d[120, 20, 80] = 17.096001
  d3d[120, 80, 20] = 17.096001

--- 3. Direct computation vs map lookup for known chords ---

Chord                        α       β       γ   Map val    Direct  Match?
---------------------------------------------------------------------------
12-TET maj7             1.2599  1.4983  1.8877   13.7181   13.4394       ✓
12-TET min7             1.1892  1.4983  1.7818   13.4701   13.1944       ✓
12-TET dom7             1.2599  1.49

In [117]:
# ============================================================================
# MAP ALL CHORDS WITH DISSONANCE VALUES (using direct Plomp-Levelt)
# ============================================================================

def map_chords_to_eigenspace(
    chords: List[Tuple[str, float, float, float]],
    system_name: str = "53-TET"
) -> pd.DataFrame:
    """
    Map chord positions to EigenSpace and compute exact dissonance values.
    
    Uses direct Plomp-Levelt computation (not the pre-computed 150³ map).
    
    Args:
        chords: List of (name, α, β, γ) tuples
        system_name: Name of the tuning system
        
    Returns:
        DataFrame with chord positions and dissonance values
    """
    records = []
    
    for name, alpha, beta, gamma in chords:
        in_tetrahedron = alpha <= beta <= gamma
        
        # Direct computation — exact at any ratio
        diss = compute_dissonance(alpha, beta, gamma)
        
        alpha_cents = 1200 * np.log2(alpha) if alpha > 0 else 0
        beta_cents = 1200 * np.log2(beta) if beta > 0 else 0
        gamma_cents = 1200 * np.log2(gamma) if gamma > 0 else 0
        
        records.append({
            'name': name,
            'system': system_name,
            'alpha': alpha,
            'beta': beta,
            'gamma': gamma,
            'alpha_cents': alpha_cents,
            'beta_cents': beta_cents,
            'gamma_cents': gamma_cents,
            'dissonance': diss,
            'in_tetrahedron': in_tetrahedron
        })
    
    return pd.DataFrame(records)


# Map both 53-TET and 12-TET chords
df_53tet = map_chords_to_eigenspace(chords_53tet, "53-TET")
df_12tet = map_chords_to_eigenspace(chords_12tet, "12-TET")

df_all_chords = pd.concat([df_53tet, df_12tet], ignore_index=True)

print(f"✓ Mapped {len(df_53tet)} 53-TET chords (direct Plomp-Levelt)")
print(f"✓ Mapped {len(df_12tet)} 12-TET chords")
print(f"✓ Total: {len(df_all_chords)} chords\n")

print("Dissonance Statistics by System:")
print(df_all_chords.groupby('system')['dissonance'].describe())

# Show top-10 most consonant 53-TET chords
print("\n\nTop 10 most consonant 53-TET chords:")
top10 = df_53tet.nsmallest(10, 'dissonance')
for _, row in top10.iterrows():
    print(f"  {row['name']:12s}  D={row['dissonance']:.4f}  α={row['alpha']:.5f} β={row['beta']:.5f} γ={row['gamma']:.5f}")

✓ Mapped 152 53-TET chords (direct Plomp-Levelt)
✓ Mapped 10 12-TET chords
✓ Total: 162 chords

Dissonance Statistics by System:
        count       mean       std        min        25%        50%  \
system                                                                
12-TET   10.0  14.967332  1.328155  13.194435  13.762464  15.149229   
53-TET  152.0  15.408889  1.204929  11.587120  14.654882  15.321717   

              75%        max  
system                        
12-TET  15.779518  16.742661  
53-TET  16.377921  18.146794  


Top 10 most consonant 53-TET chords:
  ^m^m7         D=11.5871  α=1.20093 β=1.49994 γ=1.80132
  vMvM7         D=11.8554  α=1.24898 β=1.49994 γ=1.87340
  7sus4         D=13.0561  α=1.33339 β=1.49994 γ=1.77792
  m7            D=13.2874  α=1.18533 β=1.49994 γ=1.77792
  vM^m7         D=13.3802  α=1.24898 β=1.49994 γ=1.80132
  ^mm7          D=13.4063  α=1.20093 β=1.49994 γ=1.77792
  nn7           D=13.5257  α=1.21674 β=1.49994 γ=1.82504
  SMSM7         D=13.556

## Test Case: "Something" (Beatles) — MIDI Data → EigenSpace

Parse the actual 53-TET MPE MIDI file for "Something" (type_0_major variant).  
For each chord: extract notes from MIDI+pitch bend, normalize to a single octave,
identify the third/fifth/seventh intervals, compute (α, β, γ) ratios, and look up dissonance.

**Rules:**
- Normalize all notes to one octave relative to root (mod 53)
- Discard 9th and higher extensions (keep only intervals < 53 steps)
- Map intervals to third / fifth / seventh bins
- For triads (no 7th), set γ = 2.0 (octave doubling)
- Compute dissonance via trilinear interpolation in the pre-computed map

In [118]:
# ============================================================================
# PARSE "Something" MIDI → 53-TET CHORDS (with root frequency)
# ============================================================================

import mido

MIDI_PATH = "../dataset/midi_files/53_tet_mpe/47832_Something_C_major_type_0_major.mid"

def midi_bend_to_53tet_step(midi_note: int, pitch_bend: int) -> int:
    """Convert MIDI note + pitch bend (±2 semitone range) to absolute 53-TET step."""
    bend_semitones = (pitch_bend / 8192) * 2  # ±2 semitones MPE range
    true_semitone = midi_note + bend_semitones
    return round(true_semitone * (53 / 12))


def parse_mpe_midi_chords(midi_path: str) -> List[Dict]:
    """
    Parse an MPE MIDI file into a list of chord events.
    
    Each chord carries its ROOT FREQUENCY — the origin of its EigenSpace vector.
    
    Returns list of dicts:
        {
            'beat': float,           # onset in beats
            'steps_53': List[int],   # absolute 53-TET steps per note
            'root_step': int,        # absolute 53-TET step of root (lowest note)
            'root_hz': float,        # root frequency in Hz
            'intervals': List[int],  # intervals mod 53 relative to root (sorted, unique)
        }
    """
    mid = mido.MidiFile(midi_path)
    tpb = mid.ticks_per_beat
    track = mid.tracks[1]  # notes are always on track 1

    # Collect all timestamped events
    ticks = 0
    events = []
    for msg in track:
        ticks += msg.time
        if not msg.is_meta:
            events.append((ticks, msg))

    # Group note_on events by onset tick
    channel_bend = {}
    chords_raw = []
    current_tick = None
    current_notes = []

    for tick, msg in events:
        if msg.type == 'pitchwheel':
            channel_bend[msg.channel] = msg.pitch
        elif msg.type == 'note_on' and msg.velocity > 0:
            bend = channel_bend.get(msg.channel, 0)
            if current_tick is None or tick != current_tick:
                if current_notes:
                    chords_raw.append((current_tick, list(current_notes)))
                current_notes = []
                current_tick = tick
            current_notes.append((msg.note, bend))

    if current_notes:
        chords_raw.append((current_tick, list(current_notes)))

    # Convert to 53-TET steps and compute intervals
    chords = []
    for tick, notes in chords_raw:
        steps = sorted([midi_bend_to_53tet_step(n, b) for n, b in notes])
        root_step = min(steps)
        root_hz = step53_to_hz(root_step)
        intervals = sorted(set((s - root_step) % 53 for s in steps))
        chords.append({
            'beat': tick / tpb,
            'steps_53': steps,
            'root_step': root_step,
            'root_hz': root_hz,
            'intervals': intervals,
        })

    return chords


raw_chords = parse_mpe_midi_chords(MIDI_PATH)
print(f"Parsed {len(raw_chords)} chords from MIDI\n")

# Show first 15 with root frequency
print(f"  {'#':>3} {'Beat':>6}  {'Root':>5} {'Root Hz':>8}  Intervals")
print("-" * 70)
for i, ch in enumerate(raw_chords[:15]):
    print(f"  {i+1:3d} {ch['beat']:6.1f}  {ch['root_step']:>5d} {ch['root_hz']:>8.1f}  {ch['intervals']}")

# Show root frequency range for the whole song
root_freqs = [ch['root_hz'] for ch in raw_chords]
print(f"\nRoot frequency range: {min(root_freqs):.1f} Hz – {max(root_freqs):.1f} Hz")
print(f"  (that's {1200 * np.log2(max(root_freqs)/min(root_freqs)):.0f} cents span)")

Parsed 165 chords from MIDI

    #   Beat   Root  Root Hz  Intervals
----------------------------------------------------------------------
    1    0.0    234    174.4  [0, 18, 31]
    2    4.0    226    157.1  [0, 17, 30]
    3    6.0    221    147.2  [0, 13, 22, 40]
    4    8.0    212    130.8  [0, 18, 31]
    5   16.0    212    130.8  [0, 18, 31, 49]
    6   24.0    212    130.8  [0, 18, 31, 44]
    7   32.0    181     87.2  [0, 18, 31]
    8   38.0    177     82.8  [0, 4, 22, 35]
    9   40.0    168     73.6  [0, 17, 31, 44]
   10   48.0    190     98.1  [0, 18, 31, 44]
   11   50.0    199    110.4  [0, 9, 22, 35, 44]
   12   52.0    208    124.1  [0, 13, 26, 35]
   13   56.0    199    110.4  [0, 13, 31]
   14   60.0    199    110.4  [0, 13, 31, 49]
   15   64.0    190     98.1  [0, 9, 22, 40]

Root frequency range: 65.4 Hz – 174.4 Hz
  (that's 1698 cents span)


In [119]:
# ============================================================================
# CHORD INTERVAL CLASSIFICATION → EigenSpace (α, β, γ)
# ============================================================================
# 
# Interval bins from 53tet_intervals.json:
#   Thirds:  11–20 steps  (sm=11, vm=12, m=13, ^m=14, n=15, N=16, vM=17, M=18, ^M=19, SM=20)
#   Fifths:  29–33 steps  (subdim=29, dim/vP=30, P=31, ^P/aug=32, upaug=33)
#   Sevenths: 42–51 steps (sm=42, vm=43, m=44, ^m=45, n=46, N=47, vM=48, M=49, ^M=50, SM=51)
#
# Extended range for edge cases:
#   4th (sus): 22 steps  → treat as "fourth" replacing third
#   Tritone:  26-28 steps → dim fifth region
#   6th:      35-41 steps → between fifth and seventh (drop to 9th territory or treat as 6th)

# Interval classification boundaries
THIRD_RANGE  = range(10, 23)   # 10–22 steps (includes sus4=22)
FIFTH_RANGE  = range(26, 36)   # 26–35 steps (includes tritone & augmented)
SEVENTH_RANGE = range(40, 53)  # 40–52 steps


def classify_chord_intervals(intervals: List[int]) -> Dict:
    """
    Given a list of 53-TET intervals [0, ...], classify them into
    root / third / fifth / seventh and return the EigenSpace coordinates.
    
    Intervals ≥ 53 are extensions (9ths, 11ths, 13ths) — discarded.
    
    Returns:
        {
            'third':  int or None,   # 53-TET steps
            'fifth':  int or None,
            'seventh': int or None,
            'alpha':  float,         # frequency ratio of 2nd note
            'beta':   float,         # frequency ratio of 3rd note
            'gamma':  float,         # frequency ratio of 4th note
            'chord_type': str,       # description (e.g. "M-P-m7")
            'is_triad': bool,
            'extras': List[int],     # unclassified intervals
        }
    """
    # Filter out root (0) and octave+ extensions
    iv = [x for x in intervals if 0 < x < 53]
    
    third = None
    fifth = None
    seventh = None
    extras = []
    
    for step in iv:
        if step in THIRD_RANGE and third is None:
            third = step
        elif step in FIFTH_RANGE and fifth is None:
            fifth = step
        elif step in SEVENTH_RANGE and seventh is None:
            seventh = step
        else:
            extras.append(step)
    
    # Build chord type label
    THIRD_NAMES = {
        11: "sm", 12: "vm", 13: "m", 14: "^m", 15: "n",
        16: "N", 17: "vM", 18: "M", 19: "^M", 20: "SM",
        21: "SM+", 22: "sus4"
    }
    FIFTH_NAMES = {
        26: "subdim-", 27: "subdim", 28: "subdim+", 29: "subdim",
        30: "dim", 31: "P", 32: "aug", 33: "upaug",
        34: "upaug+", 35: "upaug++"
    }
    SEVENTH_NAMES = {
        40: "sm-", 41: "sm", 42: "sm", 43: "vm", 44: "m", 45: "^m",
        46: "n", 47: "N", 48: "vM", 49: "M", 50: "^M", 51: "SM", 52: "SM+"
    }
    
    t_name = THIRD_NAMES.get(third, f"?{third}") if third else "–"
    f_name = FIFTH_NAMES.get(fifth, f"?{fifth}") if fifth else "–"
    s_name = SEVENTH_NAMES.get(seventh, f"?{seventh}") if seventh else "–"
    chord_type = f"{t_name}-{f_name}-{s_name}"
    
    # Compute frequency ratios
    alpha = get_53tet_ratio(third) if third else 1.0
    beta  = get_53tet_ratio(fifth) if fifth else 1.0
    gamma = get_53tet_ratio(seventh) if seventh else 2.0  # octave for triads
    
    is_triad = seventh is None
    
    return {
        'third': third,
        'fifth': fifth,
        'seventh': seventh,
        'alpha': alpha,
        'beta': beta,
        'gamma': gamma,
        'chord_type': chord_type,
        'is_triad': is_triad,
        'extras': extras,
    }


# Test on first few chords
print("Interval classification test:")
print("=" * 90)
for i, ch in enumerate(raw_chords[:10]):
    cl = classify_chord_intervals(ch['intervals'])
    status = "TRIAD" if cl['is_triad'] else "TETRAD"
    extra_str = f"  extras={cl['extras']}" if cl['extras'] else ""
    print(
        f"  Chord {i+1:2d} (beat {ch['beat']:5.1f}): "
        f"iv={str(ch['intervals']):30s} → {cl['chord_type']:16s} "
        f"α={cl['alpha']:.5f} β={cl['beta']:.5f} γ={cl['gamma']:.5f}  "
        f"[{status}]{extra_str}"
    )

Interval classification test:
  Chord  1 (beat   0.0): iv=[0, 18, 31]                    → M-P-–            α=1.26543 β=1.49994 γ=2.00000  [TRIAD]
  Chord  2 (beat   4.0): iv=[0, 17, 30]                    → vM-dim-–         α=1.24898 β=1.48045 γ=2.00000  [TRIAD]
  Chord  3 (beat   6.0): iv=[0, 13, 22, 40]                → m-–-sm-          α=1.18533 β=1.00000 γ=1.68730  [TETRAD]  extras=[22]
  Chord  4 (beat   8.0): iv=[0, 18, 31]                    → M-P-–            α=1.26543 β=1.49994 γ=2.00000  [TRIAD]
  Chord  5 (beat  16.0): iv=[0, 18, 31, 49]                → M-P-M            α=1.26543 β=1.49994 γ=1.89806  [TETRAD]
  Chord  6 (beat  24.0): iv=[0, 18, 31, 44]                → M-P-m            α=1.26543 β=1.49994 γ=1.77792  [TETRAD]
  Chord  7 (beat  32.0): iv=[0, 18, 31]                    → M-P-–            α=1.26543 β=1.49994 γ=2.00000  [TRIAD]
  Chord  8 (beat  38.0): iv=[0, 4, 22, 35]                 → sus4-upaug++-–   α=1.33339 β=1.58050 γ=2.00000  [TRIAD]  extras=[4]
  Chor

In [120]:
# ============================================================================
# MAP ALL CHORDS → EigenSpace + DISSONANCE (per-root frequency)
# ============================================================================

def map_song_to_eigenspace(chords: List[Dict]) -> pd.DataFrame:
    """
    Map a full song's chord list to EigenSpace 4D coordinates.
    
    Each chord's dissonance D is computed from ITS OWN ROOT FREQUENCY.
    (α, β, γ) are ratio vectors from root. D is the Plomp-Levelt dissonance
    at the chord's actual register — this is the correct EigenSpace 4D coord.
    
    Args:
        chords: Output of parse_mpe_midi_chords() — must include root_hz
        
    Returns:
        DataFrame with: beat, intervals, chord_type, alpha, beta, gamma,
        dissonance, root_hz, root_step, is_triad, in_tetrahedron
    """
    rows = []
    for ch in chords:
        cl = classify_chord_intervals(ch['intervals'])
        
        in_tetrahedron = cl['alpha'] <= cl['beta'] <= cl['gamma']
        
        # Dissonance from THIS chord's actual root frequency
        diss = compute_dissonance(
            cl['alpha'], cl['beta'], cl['gamma'],
            base_freq=ch['root_hz']
        )
        
        rows.append({
            'beat': ch['beat'],
            'intervals': str(ch['intervals']),
            'chord_type': cl['chord_type'],
            'third': cl['third'],
            'fifth': cl['fifth'],
            'seventh': cl['seventh'],
            'alpha': cl['alpha'],
            'beta': cl['beta'],
            'gamma': cl['gamma'],
            'dissonance': diss,
            'root_step': ch['root_step'],
            'root_hz': ch['root_hz'],
            'is_triad': cl['is_triad'],
            'in_tetrahedron': in_tetrahedron,
            'extras': str(cl['extras']) if cl['extras'] else '',
        })
    
    return pd.DataFrame(rows)


# Map the full song
df_something = map_song_to_eigenspace(raw_chords)

print(f"'Something' — {len(df_something)} chords mapped to EigenSpace")
print(f"  Root frequency range: {df_something['root_hz'].min():.1f} – {df_something['root_hz'].max():.1f} Hz")
print(f"  Triads:  {df_something['is_triad'].sum()}")
print(f"  Tetrads: {(~df_something['is_triad']).sum()}")
print(f"  Dissonance range: {df_something['dissonance'].min():.3f} – {df_something['dissonance'].max():.3f}")
print()

# Show unique chord types, sorted by average dissonance
unique_types = df_something.drop_duplicates(subset='chord_type').sort_values('dissonance')
print(f"Unique chord voicing types: {len(unique_types)}")
print("=" * 120)
print(f"  {'Type':18s} {'3rd':>4s} {'5th':>4s} {'7th':>4s}   "
      f"{'α':>8s} {'β':>8s} {'γ':>8s}   {'D':>8s}  {'Root Hz':>8s}  {'Tet?':>4s}  Cnt")
print("-" * 120)

for _, row in unique_types.iterrows():
    count = (df_something['chord_type'] == row['chord_type']).sum()
    tet = "✓" if row['in_tetrahedron'] else "✗"
    print(
        f"  {row['chord_type']:18s} {row['third'] or '–':>4} {row['fifth'] or '–':>4} {row['seventh'] or '–':>4}   "
        f"{row['alpha']:8.5f} {row['beta']:8.5f} {row['gamma']:8.5f}   "
        f"{row['dissonance']:8.3f}  {row['root_hz']:8.1f}  {tet:>4s}  {count}"
    )

'Something' — 165 chords mapped to EigenSpace
  Root frequency range: 65.4 – 174.4 Hz
  Triads:  99
  Tetrads: 66
  Dissonance range: 13.989 – 41.928

Unique chord voicing types: 16
  Type                3rd  5th  7th          α        β        γ          D   Root Hz  Tet?  Cnt
------------------------------------------------------------------------------------------------------------------------
  M-P-–                18 31.0  nan    1.26543  1.49994  2.00000     13.989     174.4     ✓  36
  vM-dim-–             17 30.0  nan    1.24898  1.48045  2.00000     16.510     157.1     ✓  12
  m-–-sm-              13  nan 40.0    1.18533  1.00000  1.68730     19.742     147.2     ✗  12
  M-P-M                18 31.0 49.0    1.26543  1.49994  1.89806     20.179     130.8     ✓  6
  vM-P-–               17 31.0  nan    1.24898  1.49994  2.00000     20.364     110.4     ✓  21
  m-P-–                13 31.0  nan    1.18533  1.49994  2.00000     20.967     110.4     ✓  6
  m-subdim--–          13 

In [121]:
# ============================================================================
# FULL PROGRESSION TABLE — beat-by-beat with root frequency
# ============================================================================

print("'Something' (Beatles) — Full Chord Progression in EigenSpace")
print("=" * 130)
print(f"  {'#':>3s}  {'Beat':>6s}  {'Root Hz':>8s}  {'Intervals':24s}  {'Type':18s}  "
      f"{'α':>8s}  {'β':>8s}  {'γ':>8s}  {'D':>8s}  Ext")
print("-" * 130)

for i, row in df_something.iterrows():
    extras = row['extras'] if row['extras'] else ""
    marker = " ◄" if row['is_triad'] else ""
    print(
        f"  {i+1:3d}  {row['beat']:6.1f}  {row['root_hz']:8.1f}  {row['intervals']:24s}  {row['chord_type']:18s}  "
        f"{row['alpha']:8.5f}  {row['beta']:8.5f}  {row['gamma']:8.5f}  "
        f"{row['dissonance']:8.3f}  {extras}{marker}"
    )

'Something' (Beatles) — Full Chord Progression in EigenSpace
    #    Beat   Root Hz  Intervals                 Type                       α         β         γ         D  Ext
----------------------------------------------------------------------------------------------------------------------------------
    1     0.0     174.4  [0, 18, 31]               M-P-–                1.26543   1.49994   2.00000    13.989   ◄
    2     4.0     157.1  [0, 17, 30]               vM-dim-–             1.24898   1.48045   2.00000    16.510   ◄
    3     6.0     147.2  [0, 13, 22, 40]           m-–-sm-              1.18533   1.00000   1.68730    19.742  [22]
    4     8.0     130.8  [0, 18, 31]               M-P-–                1.26543   1.49994   2.00000    18.120   ◄
    5    16.0     130.8  [0, 18, 31, 49]           M-P-M                1.26543   1.49994   1.89806    20.179  
    6    24.0     130.8  [0, 18, 31, 44]           M-P-m                1.26543   1.49994   1.77792    22.673  
    7    32

In [122]:
# ============================================================================
# 3D EIGENSPACE VISUALIZATION
# ============================================================================
# Replicates the visualization from Harmonic_Eigenspace.js:
#   - Color scale from the 150³ dissonance MAP (percentiles of the field)
#   - 53-TET chords as positioned dots, colored by their map dissonance
#   - "Something" chords overlaid, same color scale
#   - All dissonance at 220 Hz / 6 harmonics (the map reference)

# ── Color scale from the dissonance MAP ─────────────────────────────────────
# JS:  vmin = percentile(dData, 5);  vmax = percentile(dData, 95);
# dissonance_4D.ipynb:  vmin = np.percentile(dissonance_3d, 5);  vmax = np.percentile(dissonance_3d, 80)
# Both converge on ≈14.21 / ≈19.70 in the stored outputs. Using p5/p80 to match.
D_MAP_MIN = float(np.percentile(dissonance_3d, 5))
D_MAP_MAX = float(np.percentile(dissonance_3d, 80))

print(f"EigenSpace color scale (from 150³ map at 220 Hz):")
print(f"  cmin = {D_MAP_MIN:.4f}  (percentile 5)")
print(f"  cmax = {D_MAP_MAX:.4f}  (percentile 80)")
print(f"  Map absolute range: {dissonance_3d.min():.4f} – {dissonance_3d.max():.4f}")

# JS colorscale: blue → cyan → white → yellow → red
eigenspace_colorscale = [
    [0.00, "rgba(0,   0, 255, 1.0)"],
    [0.25, "rgba(0, 200, 255, 1.0)"],
    [0.50, "rgba(255,255,255, 1.0)"],
    [0.75, "rgba(255,200,  0, 1.0)"],
    [1.00, "rgba(255,  0,  0, 1.0)"],
]

# ── Prepare data ────────────────────────────────────────────────────────────
# 53-TET chords: already computed at 220 Hz in df_53tet
df_ref = df_53tet[df_53tet['in_tetrahedron']].copy()

# "Something" chords: compute D at 220 Hz (same reference as the map)
df_song = df_something[df_something['in_tetrahedron'] & df_something['dissonance'].notna()].copy()
df_song['d_220'] = [
    compute_dissonance(row['alpha'], row['beta'], row['gamma'], base_freq=220.0)
    for _, row in df_song.iterrows()
]

# ── Match each song chord to nearest 53-TET chord name ──────────────────────
ref_pos = df_ref[['alpha', 'beta', 'gamma']].values   # (N_ref, 3)
song_pos = df_song[['alpha', 'beta', 'gamma']].values  # (N_song, 3)

# Euclidean distance in (α, β, γ) space: (N_song, N_ref)
dists = np.sqrt(((song_pos[:, None, :] - ref_pos[None, :, :]) ** 2).sum(axis=2))
nearest_idx = dists.argmin(axis=1)
df_song['matched_name'] = df_ref['name'].values[nearest_idx]
df_song['match_dist'] = dists.min(axis=1)

print(f"\n53-TET chords (in tetrahedron): {len(df_ref)}")
print(f"  D range: {df_ref['dissonance'].min():.3f} – {df_ref['dissonance'].max():.3f}")
print(f"'Something' chords (in tetrahedron): {len(df_song)}")
print(f"  D at 220 Hz: {df_song['d_220'].min():.3f} – {df_song['d_220'].max():.3f}")
print(f"  Matched chord names (top 10): {df_song['matched_name'].value_counts().head(10).to_dict()}")
print(f"  Max match distance: {df_song['match_dist'].max():.6f}")

# ── Build figure ────────────────────────────────────────────────────────────
fig = go.Figure()

# Layer 1: 53-TET reference chords (circles, like JS)
fig.add_trace(go.Scatter3d(
    x=df_ref['alpha'], y=df_ref['beta'], z=df_ref['gamma'],
    mode='markers+text',
    marker=dict(
        size=5,
        color=df_ref['dissonance'],
        colorscale=eigenspace_colorscale,
        cmin=D_MAP_MIN,
        cmax=D_MAP_MAX,
        symbol='circle',
        opacity=0.9,
        line=dict(width=0.5, color='gray'),
    ),
    text=df_ref['name'],
    textposition='top center',
    textfont=dict(size=7, color='rgba(100,100,100,0.6)', family='Source Code Pro'),
    customdata=[
        f"{row['name']} (D: {row['dissonance']:.3f})"
        for _, row in df_ref.iterrows()
    ],
    hovertemplate=(
        '<b>%{customdata}</b><br>'
        'α = %{x:.4f}<br>β = %{y:.4f}<br>γ = %{z:.4f}<extra></extra>'
    ),
    name=f'53-TET Chords ({len(df_ref)})',
    showlegend=True,
))

# Layer 2: Song trajectory line
fig.add_trace(go.Scatter3d(
    x=df_song['alpha'], y=df_song['beta'], z=df_song['gamma'],
    mode='lines',
    line=dict(color='rgba(80, 80, 80, 0.3)', width=2),
    name='Song trajectory',
    showlegend=True,
    hoverinfo='skip',
))

# Layer 3: "Something" chords — same color scale
fig.add_trace(go.Scatter3d(
    x=df_song['alpha'], y=df_song['beta'], z=df_song['gamma'],
    mode='markers',
    marker=dict(
        size=6,
        color=df_song['d_220'],
        colorscale=eigenspace_colorscale,
        cmin=D_MAP_MIN,
        cmax=D_MAP_MAX,
        colorbar=dict(
            title='Dissonance (220 Hz)',
            thickness=15, len=0.6,
            tickfont=dict(size=10, family='Source Code Pro'),
            tickformat='.1f',
        ),
        symbol='diamond',
        opacity=0.95,
        line=dict(width=1, color='black'),
    ),
    customdata=[
        f"{row['matched_name']} (D: {row['d_220']:.3f})"
        for _, row in df_song.iterrows()
    ],
    text=[
        f"{row['matched_name']}<br>"
        f"α={row['alpha']:.4f} β={row['beta']:.4f} γ={row['gamma']:.4f}<br>"
        f"D={row['d_220']:.3f}"
        for _, row in df_song.iterrows()
    ],
    hoverinfo='text',
    name=f'"Something" ({len(df_song)} chords)',
    showlegend=True,
))

fig.update_layout(
    title=dict(
        text="EigenSpace — 53-TET Chords + 'Something' Trajectory<br>"
             f"<sub>Color: map p5/p80 = [{D_MAP_MIN:.2f}, {D_MAP_MAX:.2f}] · 220 Hz · 6 harmonics</sub>",
        font=dict(size=14, family='Source Code Pro'),
    ),
    scene=dict(
        xaxis=dict(title='α (2nd note ratio)', range=[1.0, 2.0],
                   backgroundcolor='white', gridcolor='lightgray',
                   tickfont=dict(family='Source Code Pro', size=9)),
        yaxis=dict(title='β (3rd note ratio)', range=[1.0, 2.0],
                   backgroundcolor='white', gridcolor='lightgray',
                   tickfont=dict(family='Source Code Pro', size=9)),
        zaxis=dict(title='γ (4th note ratio)', range=[1.0, 2.0],
                   backgroundcolor='white', gridcolor='lightgray',
                   tickfont=dict(family='Source Code Pro', size=9)),
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.0)),
        bgcolor='white',
        aspectmode='cube',
    ),
    width=1000, height=800,
    paper_bgcolor='white',
    legend=dict(
        x=0.02, y=0.98,
        font=dict(size=11, family='Source Code Pro'),
        bgcolor='rgba(255,255,255,0.8)',
    ),
)

fig.show()

print(f"\nColor scale: [{D_MAP_MIN:.2f}, {D_MAP_MAX:.2f}] from 150³ EigenSpace map")

EigenSpace color scale (from 150³ map at 220 Hz):
  cmin = 14.2065  (percentile 5)
  cmax = 19.6967  (percentile 80)
  Map absolute range: 0.6725 – 30.6237

53-TET chords (in tetrahedron): 152
  D range: 11.587 – 18.147
'Something' chords (in tetrahedron): 141
  D at 220 Hz: 10.429 – 15.661
  Matched chord names (top 10): {'MSM7': 36, 'vMSM7': 33, 'Mm7': 12, 'M+S7': 12, 'vMm7': 12, 'maj7': 6, 'M+m7': 6, 'øS7': 6, 'mSM7': 6, 'mmaj7': 6}
  Max match distance: 0.085351



Color scale: [14.21, 19.70] from 150³ EigenSpace map


In [123]:
# ============================================================================
# DISSONANCE TIMELINE — 'Something' tension evolution
# ============================================================================
# Color and Y-axis from the EigenSpace map scale (p5/p80)

df_tl = df_something[df_something['dissonance'].notna()].copy()
df_tl['d_220'] = [
    compute_dissonance(row['alpha'], row['beta'], row['gamma'], base_freq=220.0)
    for _, row in df_tl.iterrows()
]

fig2 = go.Figure()

# Reference band: map scale
fig2.add_hrect(
    y0=D_MAP_MIN, y1=D_MAP_MAX,
    fillcolor='rgba(180, 200, 255, 0.12)',
    line_width=0,
    annotation_text=f'Map p5–p80 [{D_MAP_MIN:.1f}, {D_MAP_MAX:.1f}]',
    annotation_position='top left',
    annotation=dict(font=dict(size=9, color='gray', family='Source Code Pro')),
)

# Song dissonance at 220 Hz
fig2.add_trace(go.Scatter(
    x=df_tl['beat'],
    y=df_tl['d_220'],
    mode='lines+markers',
    marker=dict(
        size=5,
        color=df_tl['d_220'],
        colorscale=eigenspace_colorscale,
        cmin=D_MAP_MIN,
        cmax=D_MAP_MAX,
        line=dict(width=0.5, color='black'),
        colorbar=dict(
            title='D (220 Hz)',
            thickness=12, len=0.5,
            x=1.05,
            tickfont=dict(size=9, family='Source Code Pro'),
            tickformat='.1f',
        ),
    ),
    line=dict(color='gray', width=1),
    text=[
        f"{row['chord_type']}<br>D={row['d_220']:.3f}"
        for _, row in df_tl.iterrows()
    ],
    hoverinfo='text',
    name='D at 220 Hz',
))

# Smoothed trend
window = 5
if len(df_tl) > window:
    smoothed = df_tl['d_220'].rolling(window, center=True).mean()
    fig2.add_trace(go.Scatter(
        x=df_tl['beat'], y=smoothed,
        mode='lines',
        line=dict(color='rgba(255, 0, 0, 0.4)', width=3),
        name=f'Smoothed (w={window})',
    ))

fig2.update_layout(
    title=dict(
        text="'Something' — Dissonance Timeline<br>"
             f"<sub>D at 220 Hz · map scale [{D_MAP_MIN:.1f}, {D_MAP_MAX:.1f}]</sub>",
        font=dict(size=14, family='Source Code Pro'),
    ),
    xaxis=dict(title='Beat', gridcolor='lightgray'),
    yaxis=dict(
        title='Dissonance D',
        gridcolor='lightgray',
        range=[D_MAP_MIN - 1.5, D_MAP_MAX + 1.5],
    ),
    width=1200, height=400,
    paper_bgcolor='white', plot_bgcolor='white',
    legend=dict(x=0.02, y=0.98, font=dict(family='Source Code Pro', size=11)),
)

fig2.show()

print(f"\n'Something' D at 220 Hz:  {df_tl['d_220'].min():.3f} – {df_tl['d_220'].max():.3f}  (mean {df_tl['d_220'].mean():.3f})")
print(f"Map scale: [{D_MAP_MIN:.3f}, {D_MAP_MAX:.3f}]")


'Something' D at 220 Hz:  5.762 – 15.661  (mean 12.367)
Map scale: [14.207, 19.697]


# Pipeline Validation: `midi_to_eigenspace()` — Any Song → 4D Coordinates

The complete method that will be integrated into the transformer's positional embedding:

```
MPE MIDI → parse chords → classify intervals → (α, β, γ) ratios
                                                       ↓
                                              compute D at root Hz (Plomp-Levelt)
                                                       ↓
                                        match to nearest 53-TET chord name
                                                       ↓
                                              (α, β, γ, D) per chord
                                                       ↓
                                           per-token embedding in GPT-2
```

**Key difference from the current `eigenspace.py`**: Dissonance D is computed with the **actual root frequency** of each chord (not a fixed 220 Hz map lookup). Low-register chords have wider critical bands → more dissonance. This is the physics.

In [124]:
# ============================================================================
# UNIFIED PIPELINE: Any MPE MIDI → EigenSpace 4D Coordinates
# ============================================================================
# This is the COMPLETE, CORRECT method for mapping any song to EigenSpace.
# It will replace the map-lookup approach in eigenspace.py for the transformer.

def midi_to_eigenspace(midi_path: str,
                       ref_chords: pd.DataFrame = None,
                       num_harmonics: int = 6,
                       ) -> pd.DataFrame:
    """
    Map any 53-TET MPE MIDI file to EigenSpace 4D coordinates.
    
    Pipeline:
        1. Parse MIDI → chord events (with root frequency)
        2. Classify intervals → (α, β, γ) frequency ratios
        3. Compute dissonance D via Plomp-Levelt at ACTUAL root Hz
        4. Match to nearest 53-TET chord vocabulary name
        5. Return DataFrame with full EigenSpace coordinates
    
    Args:
        midi_path:      Path to 53-TET MPE MIDI file
        ref_chords:     53-TET chord reference DataFrame (for name matching).
                        If None, generates it from get_53tet_chord_positions().
        num_harmonics:  Harmonics per voice for dissonance computation (default 6)
    
    Returns:
        DataFrame with columns:
            beat, root_hz, root_step, intervals, chord_type,
            alpha, beta, gamma,
            d_root     (D at actual root frequency — the TRUE 4th dimension),
            d_220      (D at 220 Hz — for comparison with the map),
            matched_name (nearest 53-TET chord vocabulary name),
            match_dist   (Euclidean distance to nearest chord),
            is_triad, in_tetrahedron, extras
    """
    # ── 1. Parse MIDI ───────────────────────────────────────────────────────
    raw = parse_mpe_midi_chords(midi_path)
    
    # ── 2–3. Classify intervals + compute D at root Hz ──────────────────────
    rows = []
    for ch in raw:
        cl = classify_chord_intervals(ch['intervals'])
        in_tet = cl['alpha'] <= cl['beta'] <= cl['gamma']
        
        d_root = compute_dissonance(
            cl['alpha'], cl['beta'], cl['gamma'],
            base_freq=ch['root_hz'], num_harmonics=num_harmonics
        )
        d_220 = compute_dissonance(
            cl['alpha'], cl['beta'], cl['gamma'],
            base_freq=220.0, num_harmonics=num_harmonics
        )
        
        rows.append({
            'beat':       ch['beat'],
            'root_hz':    ch['root_hz'],
            'root_step':  ch['root_step'],
            'intervals':  str(ch['intervals']),
            'chord_type': cl['chord_type'],
            'third':      cl['third'],
            'fifth':      cl['fifth'],
            'seventh':    cl['seventh'],
            'alpha':      cl['alpha'],
            'beta':       cl['beta'],
            'gamma':      cl['gamma'],
            'd_root':     d_root,
            'd_220':      d_220,
            'is_triad':   cl['is_triad'],
            'in_tetrahedron': in_tet,
            'extras':     str(cl['extras']) if cl['extras'] else '',
        })
    
    df = pd.DataFrame(rows)
    
    # ── 4. Match to nearest 53-TET chord vocabulary name ────────────────────
    if ref_chords is None:
        ref_chords = map_chords_to_eigenspace(chords_53tet, alpha_range,
                                               beta_range, gamma_range,
                                               dissonance_3d)
        ref_chords = ref_chords[ref_chords['in_tetrahedron']].copy()
    
    ref_pos = ref_chords[['alpha', 'beta', 'gamma']].values
    in_tet = df[df['in_tetrahedron']].copy()
    
    if len(in_tet) > 0:
        song_pos = in_tet[['alpha', 'beta', 'gamma']].values
        dists = np.sqrt(((song_pos[:, None, :] - ref_pos[None, :, :]) ** 2).sum(axis=2))
        nearest_idx = dists.argmin(axis=1)
        
        df.loc[in_tet.index, 'matched_name'] = ref_pos_names = ref_chords['name'].values[nearest_idx]
        df.loc[in_tet.index, 'match_dist'] = dists.min(axis=1)
    
    # Fill non-tetrahedron chords
    df['matched_name'] = df['matched_name'].fillna('?')
    df['match_dist'] = df['match_dist'].fillna(np.nan)
    
    return df


# ── Test on "Something" ────────────────────────────────────────────────────
test_path = "../dataset/midi_files/53_tet_mpe/47832_Something_C_major_type_0_major.mid"
df_test = midi_to_eigenspace(test_path, ref_chords=df_ref)

print(f"midi_to_eigenspace() — 'Something' (Beatles)")
print(f"=" * 100)
print(f"  Chords:     {len(df_test)}")
print(f"  In tetrah:  {df_test['in_tetrahedron'].sum()}")
print(f"  Triads:     {df_test['is_triad'].sum()}")
print(f"  Root Hz:    {df_test['root_hz'].min():.1f} – {df_test['root_hz'].max():.1f}")
print(f"  D@root:     {df_test['d_root'].min():.3f} – {df_test['d_root'].max():.3f}")
print(f"  D@220:      {df_test['d_220'].min():.3f} – {df_test['d_220'].max():.3f}")
print(f"  Matched:    {df_test[df_test['matched_name'] != '?']['matched_name'].nunique()} unique chord names")
print()
print(f"  Top chord names:")
for name, cnt in df_test['matched_name'].value_counts().head(10).items():
    print(f"    {name:12s}  ×{cnt}")

print(f"\n  First 10 chords:")
print(f"  {'#':>3}  {'Beat':>5}  {'Root Hz':>7}  {'Name':12}  {'α':>7}  {'β':>7}  {'γ':>7}  {'D@root':>7}  {'D@220':>7}")
print(f"  " + "-" * 80)
for i, (_, row) in enumerate(df_test.head(10).iterrows()):
    print(f"  {i+1:3d}  {row['beat']:5.1f}  {row['root_hz']:7.1f}  {row['matched_name']:12s}  "
          f"{row['alpha']:7.4f}  {row['beta']:7.4f}  {row['gamma']:7.4f}  "
          f"{row['d_root']:7.3f}  {row['d_220']:7.3f}")

midi_to_eigenspace() — 'Something' (Beatles)
  Chords:     165
  In tetrah:  141
  Triads:     99
  Root Hz:    65.4 – 174.4
  D@root:     13.989 – 41.928
  D@220:      5.762 – 15.661
  Matched:    11 unique chord names

  Top chord names:
    MSM7          ×36
    vMSM7         ×33
    ?             ×24
    Mm7           ×12
    M+S7          ×12
    vMm7          ×12
    maj7          ×6
    M+m7          ×6
    øS7           ×6
    mSM7          ×6

  First 10 chords:
    #   Beat  Root Hz  Name                α        β        γ   D@root    D@220
  --------------------------------------------------------------------------------
    1    0.0    174.4  MSM7           1.2654   1.4999   2.0000   13.989   11.592
    2    4.0    157.1  vMSM7          1.2490   1.4805   2.0000   16.510   12.850
    3    6.0    147.2  ?              1.1853   1.0000   1.6873   19.742   14.112
    4    8.0    130.8  MSM7           1.2654   1.4999   2.0000   18.120   11.592
    5   16.0    130.8  maj7         

In [126]:
# ============================================================================
# CROSS-VALIDATION: Run pipeline on 3 different songs from the dataset
# ============================================================================

test_songs = [
    ("A Night In Tunisia", "../dataset/midi_files/53_tet_mpe/10716_A Night In Tunisia_C_minor_type_0_major.mid"),
    ("My Girl",            "../dataset/midi_files/53_tet_mpe/10920_My Girl_C_major_type_0_major.mid"),
    ("So Sweet To Trust",  "../dataset/midi_files/53_tet_mpe/23897_So Sweet To Trust In Jesus_F_major_type_0_major.mid"),
]

all_results = {}
print(f"{'Song':25s}  {'Chords':>6}  {'InTet':>5}  {'RootHz':>14}  {'D@root':>16}  {'D@220':>16}  {'Names':>5}")
print("=" * 110)

for song_name, song_path in test_songs:
    try:
        df = midi_to_eigenspace(song_path, ref_chords=df_ref)
        all_results[song_name] = df
        in_tet = df['in_tetrahedron'].sum()
        print(
            f"{song_name:25s}  {len(df):6d}  {in_tet:5d}  "
            f"{df['root_hz'].min():6.1f}–{df['root_hz'].max():6.1f}  "
            f"{df['d_root'].min():7.3f}–{df['d_root'].max():7.3f}  "
            f"{df['d_220'].min():7.3f}–{df['d_220'].max():7.3f}  "
            f"{df[df['matched_name']!='?']['matched_name'].nunique():5d}"
        )
    except Exception as e:
        print(f"{song_name:25s}  ERROR: {e}")

# ── Detailed look: A Night In Tunisia ───────────────────────────────────────
if "A Night In Tunisia" in all_results:
    df_tunisia = all_results["A Night In Tunisia"]
    print(f"\n\n'A Night In Tunisia' — detailed")
    print(f"-" * 90)
    print(f"  {'#':>3}  {'Beat':>5}  {'RootHz':>7}  {'Name':12}  {'α':>7}  {'β':>7}  {'γ':>7}  {'D@root':>7}  {'D@220':>7}  {'Triad':>5}")
    print(f"  " + "-" * 85)
    for i, (_, row) in enumerate(df_tunisia[df_tunisia['in_tetrahedron']].head(20).iterrows()):
        t = "triad" if row['is_triad'] else ""
        print(f"  {i+1:3d}  {row['beat']:5.1f}  {row['root_hz']:7.1f}  {row['matched_name']:12s}  "
              f"{row['alpha']:7.4f}  {row['beta']:7.4f}  {row['gamma']:7.4f}  "
              f"{row['d_root']:7.3f}  {row['d_220']:7.3f}  {t}")

# ── Verify D@root vs D@220 divergence (the whole point) ────────────────────
print("\n\n" + "=" * 90)
print("FREQUENCY DEPENDENCE VERIFICATION")
print("=" * 90)
print("D@root vs D@220 — same (α,β,γ); different root pitch → different D")
print()
for song_name, df in all_results.items():
    tet = df[df['in_tetrahedron']].copy()
    if len(tet) == 0:
        continue
    ratio = tet['d_root'] / tet['d_220']
    print(f"  {song_name:25s}  D_root/D_220 = {ratio.min():.3f} – {ratio.max():.3f}  "
          f"(mean {ratio.mean():.3f})")
    # Correlation between root Hz and D_root/D_220
    corr = np.corrcoef(tet['root_hz'], ratio)[0, 1]
    print(f"  {'':25s}  corr(root_hz, ratio) = {corr:.3f}  "
          f"({'lower pitch → more dissonance ✓' if corr < -0.5 else 'check'})")

print("\n✓ Lower root frequencies → higher D_root relative to D_220")
print("  This is the Plomp-Levelt critical bandwidth effect.")

Song                       Chords  InTet          RootHz            D@root             D@220  Names
A Night In Tunisia            129    111    65.4– 147.2   18.120– 41.928   11.592– 17.705     10
My Girl                       162    162    65.4– 196.2    9.985– 36.256    8.821– 14.911      6
So Sweet To Trust              93     48    73.6– 155.1    9.897– 35.539    4.218– 14.213      6


'A Night In Tunisia' — detailed
------------------------------------------------------------------------------------------
    #   Beat   RootHz  Name                α        β        γ   D@root    D@220  Triad
  -------------------------------------------------------------------------------------
    1    8.0    130.8  Mm7            1.2654   1.4999   1.7779   22.673   14.911  
    2   24.0    130.8  Mm7            1.2654   1.4999   1.7779   22.673   14.911  
    3   40.0    130.8  Mm7            1.2654   1.4999   1.7779   22.673   14.911  
    4   48.0    147.2  m7             1.1853   1.4999   1.7

In [ ]:
# ============================================================================
# OUTPUT SHAPE FOR TRANSFORMER: per-chord → per-token 4D vectors
# ============================================================================
# The transformer needs a (seq_len, 4) array — one (α, β, γ, D) per token.
# midi_to_eigenspace() gives us per-CHORD coordinates.
# The tokenizer (05_midi_mpe_tokenization.py) produces CHORD_START...CHORD_END blocks.
# Every token inside a chord block inherits that chord's 4D vector.
#
# This is the BRIDGE between the notebook pipeline and eigenspace.py:
#   1. midi_to_eigenspace()  → per-chord DataFrame  (what we validated here)
#   2. EigenSpaceComputer    → per-token (seq_len, 4) array  (for the model)
#
# What needs to change in eigenspace.py:
#   - Replace DissonanceMap.lookup(α,β,γ) with compute_dissonance(α,β,γ, root_hz)
#   - Extract root_hz from the pitch tokens (min P_* step → step53_to_hz)
#   - Keep the per-token broadcast logic (CHORD_START → CHORD_END)

# ── Demonstrate the output shape ────────────────────────────────────────────
# For "A Night In Tunisia": what the model will see per chord position
df_demo = all_results["A Night In Tunisia"]
in_tet = df_demo[df_demo['in_tetrahedron']].copy()

print("EigenSpace 4D vectors — what the transformer will receive per chord position")
print("=" * 80)
print(f"  Shape per song: ({len(in_tet)}, 4)  →  broadcast to (seq_len, 4) per token\n")

# Show the 4D vector for each chord
vectors = in_tet[['alpha', 'beta', 'gamma', 'd_root']].values
print(f"  {'Chord':12s}  {'α':>8s}  {'β':>8s}  {'γ':>8s}  {'D':>8s}")
print(f"  " + "-" * 50)
for i, (_, row) in enumerate(in_tet.head(10).iterrows()):
    print(f"  {row['matched_name']:12s}  {row['alpha']:8.4f}  {row['beta']:8.4f}  {row['gamma']:8.4f}  {row['d_root']:8.3f}")

print(f"\n  ... {len(in_tet)} vectors total")
print(f"\n  D range:  {vectors[:, 3].min():.3f} – {vectors[:, 3].max():.3f}")
print(f"  D mean:   {vectors[:, 3].mean():.3f}")
print(f"  D std:    {vectors[:, 3].std():.3f}")

# ── Statistics across all validated songs ────────────────────────────────────
print("\n\nACROSS ALL VALIDATED SONGS")
print("=" * 80)
all_d_root = []
all_d_220 = []
for name, df in {**all_results, "Something": df_test}.items():
    tet = df[df['in_tetrahedron']]
    all_d_root.extend(tet['d_root'].tolist())
    all_d_220.extend(tet['d_220'].tolist())

all_d_root = np.array(all_d_root)
all_d_220 = np.array(all_d_220)

print(f"  Total chords pooled:  {len(all_d_root)}")
print(f"  D@root: mean={all_d_root.mean():.3f}, std={all_d_root.std():.3f}, range=[{all_d_root.min():.3f}, {all_d_root.max():.3f}]")
print(f"  D@220:  mean={all_d_220.mean():.3f}, std={all_d_220.std():.3f}, range=[{all_d_220.min():.3f}, {all_d_220.max():.3f}]")

print(f"\n  For z-normalization in the embedding:")
print(f"    D_mean = {all_d_root.mean():.4f}")
print(f"    D_std  = {all_d_root.std():.4f}")
print(f"    Normalized range: [{(all_d_root.min() - all_d_root.mean()) / all_d_root.std():.2f}, "
      f"{(all_d_root.max() - all_d_root.mean()) / all_d_root.std():.2f}]")